In [3]:
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
import os
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    precision_score,
    recall_score,
    f1_score,
    matthews_corrcoef
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# ================================
# 0. CREATE MODEL DIRECTORY
# ================================
os.makedirs("model", exist_ok=True)

# ================================
# 1. LOAD TFDS BEANS DATASET
# ================================
dataset = tfds.load("beans", split="train", as_supervised=True)

X = []
y = []

for image, label in tfds.as_numpy(dataset):
    # Feature extraction from images
    X.append([
        image.mean(),
        image.std(),
        image.min(),
        image.max(),
        np.median(image),
        np.percentile(image, 25),
        np.percentile(image, 75),
        image.var(),
        image.mean(axis=(0, 1)).mean(),   # channel mean
        image.std(axis=(0, 1)).mean(),    # channel std
        image.max(axis=(0, 1)).mean(),
        image.min(axis=(0, 1)).mean()
    ])
    y.append(label)

X = np.array(X)
y = np.array(y)

print("Dataset shape:", X.shape)  # >=12 features ✔

# ================================
# 2. TRAIN-TEST SPLIT
# ================================
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

# ================================
# 3. FEATURE SCALING
# ================================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler for Streamlit
joblib.dump(scaler, "model/scaler.pkl")

# ================================
# 4. MODELS
# ================================
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, multi_class="ovr"),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(
        n_estimators=100,
        random_state=42
    ),
    "XGBoost": XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        objective="multi:softprob",
        eval_metric="mlogloss",
        random_state=42
    )
}

# ================================
# 5. TRAIN, EVALUATE & SAVE
# ================================
results = []

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_prob, multi_class="ovr"),
        "Precision": precision_score(y_test, y_pred, average="weighted"),
        "Recall": recall_score(y_test, y_pred, average="weighted"),
        "F1": f1_score(y_test, y_pred, average="weighted"),
        "MCC": matthews_corrcoef(y_test, y_pred)
    })

    # Save model
    filename = name.lower().replace(" ", "_") + ".pkl"
    joblib.dump(model, f"model/{filename}")

    from sklearn.metrics import confusion_matrix, classification_report

    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

# ================================
# 6. RESULTS TABLE
# ================================
results_df = pd.DataFrame(results)
print("\nModel Performance Comparison:\n")
print(results_df.to_string(index=False))

# Save results for README
results_df.to_csv("model/results.csv", index=False)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


...

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/incomplete.6HEVCG_0.1.0/beans-test.tfrecord*...:   0%|          | 0/…

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.
Dataset shape: (1034, 12)

Training Logistic Regression...
Confusion Matrix:
[[44 18 25]
 [14 42 31]
 [21 19 45]]
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.51      0.53        87
           1       0.53      0.48      0.51        87
           2       0.45      0.53      0.48        85

    accuracy                           0.51       259
   macro avg       0.51      0.51      0.51       259
weighted avg       0.51      0.51      0.51       259


Training Decision Tree...
Confusion Matrix:
[[35 22 30]
 [24 37 26]
 [23 21 41]]
Classification Report:
              precision    recall  f1-score   support

           0       0.43      0.40      0.41        87
           1       0.46      0.43      0.44        87
           2       0.42      0.48      0.45        85

    accuracy                           0

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


Confusion Matrix:
[[44 19 24]
 [17 50 20]
 [20 10 55]]
Classification Report:
              precision    recall  f1-score   support

           0       0.54      0.51      0.52        87
           1       0.63      0.57      0.60        87
           2       0.56      0.65      0.60        85

    accuracy                           0.58       259
   macro avg       0.58      0.58      0.57       259
weighted avg       0.58      0.58      0.57       259


Training XGBoost...
Confusion Matrix:
[[40 20 27]
 [14 49 24]
 [16 17 52]]
Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.46      0.51        87
           1       0.57      0.56      0.57        87
           2       0.50      0.61      0.55        85

    accuracy                           0.54       259
   macro avg       0.55      0.54      0.54       259
weighted avg       0.55      0.54      0.54       259


Model Performance Comparison:

              Model  Accuracy